# Using netMHC predictions

In [1]:
import glob, json
import numpy as np
import pandas as pd
from scipy.stats import entropy
import sklearn.decomposition
import sklearn.manifold
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.append('..')

from lib import *
from IPython.display import display, HTML
import weblogo
#from corebio.seq import SeqList, Alphabet

/media/andreas/data/repos/peptidome/code/netmhc/../lib/main.py:419: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def mcmcsampler(x0, energy, jump, nsteps=1000, nburnin=0, nsample=1):


In [3]:
datadir_maxent = '/home/andreas/drop/data/peptidome/'
dfbinders = pd.concat(pd.read_csv(filename, sep=',') for filename in glob.glob(datadir + 'netmhc/Malaria-HLA-*.csv.gz'))

ValueError: No objects to concatenate

In [ ]:
totalpeptides = sum(len(seq) for h, seq in fasta_iter(human))
totalpeptides

In [ ]:
counter9 = count_kmers_proteome(proteome_path('Malaria'), 9)

In [ ]:
len(counter9)

In [ ]:
len(dfbinders['Peptide'].unique()), len(np.array(dfbinders['Peptide']))

In [ ]:
# 50nM = strong binding
# 500nM = weak binding
dfbinders = dfbinders[dfbinders['nM']<500]
dfbinders.head()

In [ ]:
for filename in glob.glob(datadir + 'netmhc/Human-HLA-*.csv.gz'):
    dfbinders = pd.read_csv(filename, sep=',')
    seqs = weblogo.SeqList(list(dfbinders['Peptide']), alphabet=weblogo.Alphabet(aminoacids))
    data = weblogo.LogoData.from_seqs(seqs)
    options = weblogo.LogoOptions()
    #options.logo_title = 'HLA-A0101'
    f = weblogo.LogoFormat(data, options)
    svg = weblogo.svg_formatter(data, f)
    display(HTML(svg.decode('ascii')))

In [ ]:
with open('../../data/triplet-human.json', 'r') as f:
    tripletparams = json.load(f)

In [ ]:
k = 9
loglikelihood = lambda seq, k: loglikelihood_triplet(seq, **tripletparams, k=k)
phuman = np.array([loglikelihood(seq[i:i+k], k) for h, seq in fasta_iter(human) for i in range(len(seq)-k+1) ])

In [ ]:
dfproteomes = load_proteomes()
pathogenproteomes = dfproteomes[dfproteomes['type'].isin(['parasite'])]

dfepitopes = load_iedb_tcellepitopes(human_only=True, positive_only=True, peptide_length=9)
#dfepitopes = pd.read_csv(datadir+'allhuman-iedb-epitopes.csv', header=1, usecols=(2, 9, 14))
iedbname = pathogenproteomes['iedbname'].iloc[0]
dfepitopes = dfepitopes.fillna('')
dfepitope = dfepitopes[dfepitopes['Epitope', 'Parent Species'].str.contains(iedbname)]
#dfepitope = pd.read_csv(datadir+'iedb-Malaria-MHCI-humanhost.csv', header=1, usecols=(2, 9, 14))
#dfepitope['length'] = [len(d) for d in dfepitope['Description']]

In [ ]:
epi = list(dfepitope['Epitope', 'Description'])
epi = np.unique(epi)
pepitope = np.array([loglikelihood(seq, k) for seq in epi])
#epi = list(dfepitope['Description'])
#pepitope = np.array([loglikelihood(seq[i:i+k], k) for seq in epi for i in range(len(seq)-k+1)])
pepitope = pepitope[~np.isnan(pepitope)]

path = datadir + pathogenproteomes['path'].iloc[0]
ppathogen = np.array([loglikelihood(seq[i:i+k], k) for h, seq in fasta_iter(path) for i in range(len(seq)-k+1) ])
ppathogen = ppathogen[~np.isnan(ppathogen)]

print(len(phuman), len(ppathogen), len(pepitope))

In [ ]:
preds = []
for filename in glob.glob(datadir + 'netmhc/Malaria-HLA-*.csv.gz'):
    dfbinders = pd.read_csv(filename, sep=',')
    ppredepitope = np.array([loglikelihood(seq, k) for seq in dfbinders['Peptide'].unique()])
    ppredepitope = ppredepitope[~np.isnan(ppredepitope)]
    if len(ppredepitope) > 20:
        preds.append(ppredepitope)

In [ ]:
for mhc in [False, True]:
    fig, ax = plt.subplots()
    if mhc:
        plot_histograms(preds,
                        [None]*len(preds),
                        nbins=20, lw=.5, color='k', alpha=.5,
                        xmin=-14, xmax=-9, ax=ax, step=False)
    plot_histograms([phuman, ppathogen, pepitope],
                    ['human', 'pathogen', 'epitope'],
                    nbins=20, lw=2,
                    xmin=-14, xmax=-9, ax=ax, step=False)
    ax.set_ylabel('relative frequency')
    ax.set_ylim(0.0, 0.22)
    ax.set_yticks([])
    ax.set_xlabel('$\log_{10}$ probability given human proteome statistics')
    fig.tight_layout()
    #fig.savefig('plots/likelihoodprofile-Malaria-triplet-k%i%s.png' % (k, '-mhc' if mhc else ''), dpi=300)
    if mhc:
        fig.savefig('main.png')

In [ ]:
dfepi9 = dfepitope[dfepitope['length'] == 9]

In [ ]:
len(set(dfepi9['Description'])), len(set(dfepi9['Description']) - set(dfbinders['Peptide'].unique()))

In [ ]:
with open('peptides.txt', 'w') as f:
    for item in list(dfepi9['Description']):
        f.write("%s\n" % item)

In [ ]:
!netMHC peptides.txt -xls -xlsfile peptides.csv -a HLA-A0101 -p 1

In [ ]:
pd.read_csv('peptides.csv', sep='\t', skiprows=1)